In [5]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
import duckdb
import pandas as pd
import sqlalchemy

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///my-db.duckdb

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [7]:
from uniprot_log_file_parser import db, prepare_and_insert, ua

In [8]:
!rm foo.duckdb*

In [9]:
dbc = db.get_db_connection('foo.duckdb')

In [10]:
# dbc.sql("DROP TABLE uniprotkb; DROP TABLE useragent;")

In [11]:
db.setup_tables(dbc, "uniprotkb")

In [12]:
db.restore_useragent_family(dbc, 'useragent_family_df.csv')

In [13]:
db.restore_useragent(dbc, 'useragent_df.csv')

In [14]:
dbc.sql("SELECT * FROM useragent")

┌───────┬──────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│  id   │                                              string                                              │ family_id │
│ int32 │                                             varchar                                              │   int32   │
├───────┼──────────────────────────────────────────────────────────────────────────────────────────────────┼───────────┤
│     0 │                                                                                                  │         0 │
│     1 │ "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0…  │         1 │
│     2 │ "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0…  │         2 │
│     3 │ "Mozilla/5.0 (iPod; U; CPU iPhone OS 2_1 like Mac OS X; ja-jp) AppleWebKit/525.18.1 (KHTML, li…  │         3 │
│     4 │ -                     

In [15]:
dbc.sql("SELECT * FROM useragent_family")

┌───────┬───────────────────────┬───────────────────────┬─────────┐
│  id   │        family         │         type          │  major  │
│ int32 │        varchar        │ useragent_family_type │ boolean │
├───────┼───────────────────────┼───────────────────────┼─────────┤
│     0 │ Other                 │ unknown               │ false   │
│     1 │ Edge                  │ browser               │ true    │
│     2 │ Chrome                │ browser               │ true    │
│     3 │ Mobile Safari         │ browser               │ true    │
│     4 │ Firefox               │ browser               │ true    │
│     5 │ AHC                   │ bot                   │ false   │
│     6 │ ActionExtension       │ unknown               │ false   │
│     7 │ AdsBot-Google         │ bot                   │ false   │
│     8 │ Aloha                 │ unknown               │ false   │
│     9 │ Chrome Mobile WebView │ browser               │ true    │
│     · │    ·                  │    ·          

In [16]:
log_path = "/Volumes/Developer/logs/uniprotkb/prod-0/access_log.2023-01-08.log"

In [17]:
with open(log_path, encoding="utf-8") as file:
    log_contents = file.read()
log_df, n_lines_skipped = prepare_and_insert.get_log_data_frame(log_contents, log_path)

In [16]:
n_lines_skipped

0

In [18]:
unseen_useragent_df = db.get_unseen_useragent_df(dbc, log_df)

In [19]:
unseen_useragent_df

,string,id,family
0,Mozilla/5.0 (Linux; Android 10; motorola one v...,91835,Chrome Mobile
1,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,91836,Edge
2,Mozilla/5.0 (Linux; Android 12; SM-N976Q) Appl...,91837,Chrome Mobile
3,Mozilla/5.0 (Linux; Android 12; Lenovo YT-J706...,91838,Opera
4,Mozilla/5.0 (Linux; Android 10; SM-A015G) Appl...,91839,Chrome Mobile
...,...,...,...
114,Mozilla/5.0 (Linux; U; Android 12; zh-CN; 2210...,91949,Chrome Mobile WebView
115,Mozilla/5.0 (Linux; Android 12; RMX3511) Apple...,91950,Chrome Mobile
116,Mozilla/5.0 (Linux; Android 12; CPH2399) Apple...,91951,Chrome Mobile
117,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like M...,91952,Google


In [20]:
db.insert_log_data(dbc, 'uniprotkb', log_df)

ConstraintException: Constraint Error: NOT NULL constraint failed: uniprotkb.useragent_id

In [26]:
r = dbc.sql("""
SELECT
    datetime,
    method,
    request,
    status,
    bytes,
    referrer,
    ua.id,
    ua.string,
    uaf.id,
    uaf.family
FROM
    uniprotkb AS uni
JOIN
    useragent AS ua ON uni.useragent_id = ua.id
JOIN
    useragent_family AS uaf ON uaf.id = ua.family_id
""").df()

In [22]:
dbc.sql("""
SELECT
    *
FROM
    uniprotkb
""")

┌──────────────────────────┬─────────┬─────────┬────────┬────────┬──────────┬──────────────┐
│         datetime         │ method  │ request │ status │ bytes  │ referrer │ useragent_id │
│ timestamp with time zone │ varchar │ varchar │ uint16 │ uint64 │ varchar  │    int32     │
├──────────────────────────────────────────────────────────────────────────────────────────┤
│                                          0 rows                                          │
└──────────────────────────────────────────────────────────────────────────────────────────┘

In [102]:
tr = dbc.sql("""
SELECT
    *
FROM
    uniprotkb
""").df()

In [103]:
tr

,datetime,method,request,status,bytes,referrer,useragent_id
0,2023-01-08 00:00:01+00:00,GET,/uniprotkb/Q12346,200,11428,-,78921
1,2023-01-08 00:00:01+00:00,GET,/uniprotkb/Q5XF89,200,28587,-,78921
2,2023-01-08 00:00:01+00:00,GET,/uniprotkb/C7J5M3,200,6923,-,78921
3,2023-01-08 00:00:25+00:00,GET,/uniprotkb/P56728.fasta,200,435,-,78921
4,2023-01-08 00:00:26+00:00,GET,/uniprotkb/P90122.fasta,200,192,-,78921
...,...,...,...,...,...,...,...
162398,2023-01-08 23:54:29+00:00,GET,/uniprotkb/search?cursor=19y2z6pm9i1pj1jbc4vr2...,200,21849,https://www.uniprot.org/,21937
162399,2023-01-08 23:56:56+00:00,GET,/uniprotkb/search?facets=reviewed%2Cmodel_orga...,200,24,https://www.uniprot.org/,21937
162400,2023-01-08 23:58:09+00:00,GET,/uniprotkb/S5AA40,200,6340,https://www.uniprot.org/,6887
162401,2023-01-08 23:59:29+00:00,GET,/uniprotkb/search?fields=accession%2Creviewed%...,200,49808,https://www.uniprot.org/,67343


In [138]:
db.update_useragents(dbc, unseen_items_df)

In [27]:
merged[merged["id"].isnull()]

,datetime,method,request,status,bytes,referrer,useragent,id,string


In [145]:
dbc.sql("SELECT * FROM useragent WHERE major is NULL")

┌───────────────────────────────────────────────────────────┬───────────────────────┬────────────────┬─────────┬───────┐
│                          string                           │        family         │      type      │  major  │  id   │
│                          varchar                          │        varchar        │ useragent_type │ boolean │ int32 │
├───────────────────────────────────────────────────────────┼───────────────────────┼────────────────┼─────────┼───────┤
│ Mozilla/5.0 (Linux; Android 12; LSA-AN00; HMSCore 6.9.0…  │ Chrome                │ unknown        │ NULL    │ 91835 │
│ Mozilla/5.0 (Linux; Android 11; SM-G9750) AppleWebKit/5…  │ Chrome Mobile         │ unknown        │ NULL    │ 91836 │
│ Mozilla/5.0 (Linux; Android 10; MAR-LX1A) AppleWebKit/5…  │ Chrome Mobile         │ unknown        │ NULL    │ 91837 │
│ Mozilla/5.0 (Linux; Android 11; RMX3506) AppleWebKit/53…  │ Chrome Mobile         │ unknown        │ NULL    │ 91838 │
│ Mozilla/5.0 (Linux; U; Android

In [30]:
%%sql
CREATE TYPE useragent_type AS ENUM ('browser', 'programmatic', 'bot', 'unknown');
CREATE TABLE useragent
 (
     string VARCHAR,
     family VARCHAR,
     type useragent_type,
     major BOOLEAN,
     PRIMARY KEY (string, family, type, major)
 )
INSERT INTO useragent SELECT * FROM read_csv_auto('useragent.csv');

""


In [4]:
%%time
%%sql
SELECT
  COUNT(*)
FROM
  uniprotkb,
  useragent
WHERE
  method = 'GET' AND
  status = 200 AND
  useragent.string = uniprotkb.useragent AND
  useragent.major = true AND
  useragent.type = 'browser'

CPU times: user 1min 6s, sys: 4.02 s, total: 1min 10s
Wall time: 20.2 s


,count_star()
0,86373098


In [15]:
%%sql
SELECT
  COUNT(*)
FROM
  uniprotkb,
  useragent


""


In [6]:
%%time
%%sql
df << SELECT
  uniprotkb.datetime,
  uniprotkb.method,
  uniprotkb.request,
  uniprotkb.status,
  uniprotkb.bytes,
  uniprotkb.referrer,
  uniprotkb.useragent,
  useragent.family,
  useragent.type
FROM
  uniprotkb,
  useragent
WHERE
  method = 'GET' AND
  status = 200 AND
  request ILIKE '%ftev\_%' AND
  useragent.string = uniprotkb.useragent AND
  useragent.major = true

Returning data to local variable df
CPU times: user 5min, sys: 1min 22s, total: 6min 23s
Wall time: 8min 38s


In [8]:
df

""


In [7]:
df.loc[0,'request']

KeyError: 'request'

In [127]:
log_df_merged

,datetime,method,request,status,bytes,referrer,useragent_id
0,2023-01-08 00:00:01+00:00,GET,/uniprotkb/Q12346,200,11428,-,78921.0
1,2023-01-08 00:00:01+00:00,GET,/uniprotkb/W4EE35.fasta,200,254,-,84972.0
2,2023-01-08 00:00:01+00:00,GET,/uniprotkb/Q5XF89,200,28587,-,78921.0
3,2023-01-08 00:00:01+00:00,GET,/uniprotkb/C7J5M3,200,6923,-,78921.0
4,2023-01-08 00:00:01+00:00,GET,/uniprotkb/P0AEF7,200,6098,https://www.uniprot.org/,84126.0
...,...,...,...,...,...,...,...
168208,2023-01-09 00:00:00+00:00,GET,/uniprotkb/search?fields=accession%2Creviewed%...,200,7052,https://www.uniprot.org/,67971.0
168209,2023-01-09 00:00:00+00:00,GET,/uniprotkb/search?query=BBOX1&showSingleTermMa...,200,115,https://www.uniprot.org/,72983.0
168210,2023-01-09 00:00:00+00:00,GET,/uniprotkb/search?facets=reviewed%2Cmodel_orga...,200,10105,https://www.uniprot.org/,84126.0
168211,2023-01-09 00:00:00+00:00,GET,/uniprotkb/search?format=fasta&query=DHH%20AND...,200,11360,-,91791.0
